In [459]:
import pandas as pd 
import seaborn as sb
import datetime as dt
import numpy as np

In [460]:
noise = pd.read_csv('noise_zaventem.csv', sep=';')

In [461]:
noise.start_date = pd.to_datetime(noise['start_date'], format="%d/%m/%Y")

In [462]:
noise.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24177 entries, 0 to 24176
Data columns (total 14 columns):
nmt_number    24177 non-null int64
loc_number    24177 non-null int64
start_date    24177 non-null datetime64[ns]
start_time    24177 non-null object
duration      24177 non-null int64
event_leq     24177 non-null object
event_sel     24177 non-null object
max_level     24177 non-null object
max_time      24177 non-null object
runway        15794 non-null object
sid           9596 non-null object
star          5966 non-null object
movement      15822 non-null object
fldate        15831 non-null object
dtypes: datetime64[ns](1), int64(3), object(10)
memory usage: 2.6+ MB


In [463]:
noise.event_leq = noise.event_leq.str.replace(",",".").astype(float)

In [464]:
noise.max_level = noise.max_level.str.replace(",",".").astype(float)

Check of er duplicate waardes zijn in de dataset

In [465]:
noise[noise.duplicated(['nmt_number', 'loc_number', 'start_date', 'start_time', 'duration', 'event_leq', 'event_sel', 'max_level', 'max_time', 'runway', 'sid', 'star', 'movement', 'fldate'])]

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate


Check of er rijen zijn waar wel een fldate zit maar geen movement

In [466]:
noise[(noise.fldate.notnull()) & (noise.movement.isnull())]

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate
1528,41,1,2020-02-28,11:48:42,24,65.7,"79,5",74.4,11:49:00,NaN,NIK3C,NaN,NaN,28/02/2020 11:46:00
2454,41,1,2020-03-13,06:36:13,25,68.0,"82,0",75.6,06:36:25,NaN,CIV4C,NaN,NaN,13/03/2020 06:34:00
2677,41,1,2020-03-18,06:25:44,32,67.3,"82,4",76.8,06:26:03,NaN,HELEN6C,NaN,NaN,18/03/2020 06:23:00
2760,41,1,2020-03-20,06:45:57,22,67.2,"80,6",74.9,06:46:16,NaN,HELEN6C,NaN,NaN,20/03/2020 06:43:00
7620,43,2,2020-02-25,14:20:51,50,69.2,"86,2",73.1,14:21:22,NaN,NaN,FLO25L,NaN,25/02/2020 14:22:00
8631,43,2,2020-03-03,14:13:26,30,64.6,"79,4",74.9,14:13:47,NaN,NaN,FLO25L,NaN,03/03/2020 14:14:00
9269,43,2,2020-03-06,14:09:10,88,72.7,"92,1",84.2,14:09:46,NaN,NaN,BATTY5A,NaN,06/03/2020 14:20:00
11154,44,2,2020-02-25,07:26:06,20,69.6,"82,6",74.4,07:26:15,NaN,ROUSY7C,NaN,NaN,25/02/2020 07:23:00
11790,44,2,2020-03-05,09:39:25,27,66.7,"81,0",78.9,09:39:33,NaN,SOPOK8C,NaN,NaN,05/03/2020 09:36:00


Check of er rijen zijn waar wel een fldate is maar niets anders

In [467]:
noise[(noise.sid.isnull()) & (noise.star.isnull()) & (noise.fldate.notnull()) & (noise.runway.isnull()) & (noise.movement.isnull())]

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate


Conclusie is dat de fldate bepalend is voor of er een vluchtgecorreleerde beweging was. runway/sid/star/movement is niet altijd bekend maar fldate wel.

Volgende: check voor empty rows

In [468]:
len(noise.dropna(how='all'))

24177

Conclusie: geen complete empty rows.

In [469]:
noise.head()

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate
0,40,2,2020-03-11,14:12:37,33,67.4,"82,6",72.2,14:13:00,NaN,NaN,NaN,NaN,NaN
1,40,2,2020-03-11,14:22:27,40,64.6,"80,6",68.6,14:22:56,NaN,NaN,NaN,NaN,NaN
2,40,2,2020-03-11,14:49:33,47,64.0,"80,7",69.6,14:49:54,NaN,NaN,NaN,NaN,NaN
3,40,2,2020-03-11,16:04:26,19,63.4,"76,2",67.1,16:04:33,25R,DENUT6C,NaN,D,11/03/2020 16:02:00
4,40,2,2020-03-11,16:23:19,17,64.0,"76,3",67.8,16:23:21,25R,CIV4C,NaN,D,11/03/2020 16:21:00


Nu kijken we alleen naar vlucht-gecorreleerd geluid, omdat dat ons iets zegt over de vliegbewegingen die nog gebeuren vanaf Zaventem

In [470]:
flights = noise[noise.fldate.notnull()]

In [471]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15831 entries, 3 to 24175
Data columns (total 14 columns):
nmt_number    15831 non-null int64
loc_number    15831 non-null int64
start_date    15831 non-null datetime64[ns]
start_time    15831 non-null object
duration      15831 non-null int64
event_leq     15831 non-null float64
event_sel     15831 non-null object
max_level     15831 non-null float64
max_time      15831 non-null object
runway        15794 non-null object
sid           9596 non-null object
star          5966 non-null object
movement      15822 non-null object
fldate        15831 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(8)
memory usage: 1.8+ MB


In [472]:
flights.head()

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate
3,40,2,2020-03-11,16:04:26,19,63.4,"76,2",67.1,16:04:33,25R,DENUT6C,NaN,D,11/03/2020 16:02:00
4,40,2,2020-03-11,16:23:19,17,64.0,"76,3",67.8,16:23:21,25R,CIV4C,NaN,D,11/03/2020 16:21:00
5,40,2,2020-03-11,17:24:38,20,65.0,"78,0",68.5,17:24:52,25R,CIV4C,NaN,D,11/03/2020 17:22:00
6,40,2,2020-03-11,18:04:17,21,63.2,"76,4",68.3,18:04:18,25R,HELEN6C,NaN,D,11/03/2020 18:02:00
8,40,2,2020-03-11,18:39:27,43,68.1,"84,4",73.8,18:39:43,25R,NIK3C,NaN,D,11/03/2020 18:38:00


In [473]:
flights.nmt_number.value_counts()

42    3922
3     2964
43    2863
46    2194
41    1348
40     741
47     635
44     568
45     522
48      74
Name: nmt_number, dtype: int64

De meeste vliegbewegingen komen in de buurt van NMT 42, de minste bij NMT 48. 

In [474]:
flights.runway.value_counts()

25R    8739
25L    5313
01      857
19      528
07R     196
07L     155
UNK       6
Name: runway, dtype: int64

De meeste vliegbewegingen vanaf 25R, de minste vanaf 07L (en een aantal vanaf unkown)

In [475]:
flights[flights.start_date.isnull()]

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate


In [476]:
flights[flights.max_level.isnull()]

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate


In [477]:
night = flights[(flights.start_time > "23:00") | (flights.start_time < "07:00")]

In [478]:
day = flights[(flights.start_time < "23:00") & (flights.start_time > "07:00")]

In [479]:
day.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13539 entries, 3 to 24175
Data columns (total 14 columns):
nmt_number    13539 non-null int64
loc_number    13539 non-null int64
start_date    13539 non-null datetime64[ns]
start_time    13539 non-null object
duration      13539 non-null int64
event_leq     13539 non-null float64
event_sel     13539 non-null object
max_level     13539 non-null float64
max_time      13539 non-null object
runway        13510 non-null object
sid           8018 non-null object
star          5306 non-null object
movement      13533 non-null object
fldate        13539 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(8)
memory usage: 1.5+ MB


In [480]:
day[(day.nmt_number == 41) & (day.start_date == "2020-04-09")]

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate
3073,41,1,2020-04-09,07:09:35,35,65.2,"80,6",67.6,07:09:59,25R,DENUT6C,NaN,D,09/04/2020 07:08:07
3074,41,1,2020-04-09,08:42:34,48,66.9,"83,7",71.3,08:43:00,25R,DENUT6C,NaN,D,09/04/2020 08:41:00


In [481]:
night.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2292 entries, 29 to 23867
Data columns (total 14 columns):
nmt_number    2292 non-null int64
loc_number    2292 non-null int64
start_date    2292 non-null datetime64[ns]
start_time    2292 non-null object
duration      2292 non-null int64
event_leq     2292 non-null float64
event_sel     2292 non-null object
max_level     2292 non-null float64
max_time      2292 non-null object
runway        2284 non-null object
sid           1578 non-null object
star          660 non-null object
movement      2289 non-null object
fldate        2292 non-null object
dtypes: datetime64[ns](1), float64(2), int64(3), object(8)
memory usage: 268.6+ KB


In [482]:
night[(night.nmt_number == 41) & (night.start_date == "2020-04-09")]

,nmt_number,loc_number,start_date,start_time,duration,event_leq,event_sel,max_level,max_time,runway,sid,star,movement,fldate
3060,41,1,2020-04-09,00:10:20,36,63.2,"78,8",64.9,00:10:44,25R,DENUT6C,NaN,D,09/04/2020 00:08:00
3061,41,1,2020-04-09,00:30:09,29,66.9,"81,5",69.9,00:30:24,25R,SOPOK6Z,NaN,D,09/04/2020 00:28:00
3062,41,1,2020-04-09,00:57:56,25,66.0,"80,0",69.0,00:58:11,25R,DENUT6C,NaN,D,09/04/2020 00:56:00
3063,41,1,2020-04-09,01:31:21,27,66.6,"80,9",69.6,01:31:34,25R,SOPOK6Z,NaN,D,09/04/2020 01:29:00
3064,41,1,2020-04-09,03:04:20,32,66.7,"81,8",69.5,03:04:41,25R,DENUT6C,NaN,D,09/04/2020 03:03:00
3065,41,1,2020-04-09,03:10:14,36,67.7,"83,3",71.8,03:10:36,25R,NIK3C,NaN,D,09/04/2020 03:08:00
3066,41,1,2020-04-09,03:20:57,18,64.0,"76,6",65.8,03:21:05,25R,SOPOK6Z,NaN,D,09/04/2020 03:19:00
3067,41,1,2020-04-09,03:22:34,21,64.7,"77,9",66.6,03:22:48,25R,CIV4C,NaN,D,09/04/2020 03:21:00
3068,41,1,2020-04-09,03:55:33,24,63.6,"77,4",66.1,03:55:47,25R,HELEN6C,NaN,D,09/04/2020 03:53:00
3069,41,1,2020-04-09,04:24:49,13,63.9,"75,0",66.1,04:24:50,25R,ROUSY5Z,NaN,D,09/04/2020 04:22:00


## Aantal vliegbewegingen per meetstation per dag naar maximale geluidsbelasting, overdag/snachts

In [483]:
bins = [59, 65, 70, 75, 80, 85, 90, 95, 100, np.inf]
labels = ['60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-95', '95-100', 'meer dan 100']
max_day = day.groupby(["nmt_number", "start_date", pd.cut(day['max_level'], bins=bins, labels=labels)]).size().reset_index(name="count")

In [484]:
max_day.head()

,nmt_number,start_date,max_level,count
0,3,2020-02-24,60-65,30
1,3,2020-02-24,65-70,88
2,3,2020-02-24,70-75,19
3,3,2020-02-25,60-65,37
4,3,2020-02-25,65-70,80


In [485]:
max_day[(max_day.nmt_number == 41) & (max_day.start_date == '2020-04-09')]

,nmt_number,start_date,max_level,count
325,41,2020-04-09,65-70,1
326,41,2020-04-09,70-75,1


In [486]:
max_day.to_csv('vliegbewegingen_overdag.csv', index=False, sep=';')

In [487]:
max_night = night.groupby(["nmt_number", "start_date", pd.cut(night['max_level'], bins=bins, labels=labels)]).size().reset_index(name="count")

In [488]:
max_night.head()

,nmt_number,start_date,max_level,count
0,3,2020-02-24,60-65,1
1,3,2020-02-24,65-70,7
2,3,2020-02-25,65-70,8
3,3,2020-02-25,70-75,5
4,3,2020-02-25,75-80,1


In [489]:
max_night.to_csv('vliegbewegingen_snachts.csv', index=False, sep=';')

## Gemiddeld maximaal geluid per meetstation per dag, overdag/snachts

In [490]:
overdag = day[["nmt_number", "start_date", "duration", "event_leq"]]

In [491]:
overdag.event_leq = overdag.event_leq.astype(str).str.replace('.', ',')

In [492]:
overdag.head()

,nmt_number,start_date,duration,event_leq
3,40,2020-03-11,19,"63,4"
4,40,2020-03-11,17,"64,0"
5,40,2020-03-11,20,"65,0"
6,40,2020-03-11,21,"63,2"
8,40,2020-03-11,43,"68,1"


In [493]:
overdag.to_csv('geluid_overdag.csv', index=False, sep=';')

In [494]:
snachts = night[["nmt_number", "start_date", "duration", "event_leq"]]

In [495]:
snachts.event_leq = snachts.event_leq.astype(str).str.replace('.', ',')

In [496]:
snachts.head()

,nmt_number,start_date,duration,event_leq
29,40,2020-03-11,46,"65,9"
30,40,2020-03-11,67,"65,4"
31,40,2020-03-12,42,"64,6"
32,40,2020-03-12,31,"65,6"
34,40,2020-03-12,41,"66,3"


In [497]:
snachts.to_csv('geluids_snachts.csv', index=False, sep=';')

## Aantal metingen per meetstation per dag

In [498]:
dag_aantal = overdag.groupby(['nmt_number', 'start_date']).event_leq.count().reset_index()

In [499]:
dag_aantal.to_csv('metingen_dag.csv', index=False, sep=';')

In [500]:
nacht_aantal = snachts.groupby(['nmt_number', 'start_date']).event_leq.count().reset_index()

In [501]:
nacht_aantal.to_csv('metingen_nacht.csv', index=False, sep=';')